In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split

download_path = "../vocal_patterns/data/dataset_tags.csv"
data = pd.read_csv(download_path)
data.head()

,path,exercise,technique,filename
0,/Users/jake/code/jchaselubitz/vocal_patterns/v...,Other,vibrato,m6_row_vibrato.wav
1,/Users/jake/code/jchaselubitz/vocal_patterns/v...,Other,vibrato,m6_caro_vibrato.wav
2,/Users/jake/code/jchaselubitz/vocal_patterns/v...,Other,vibrato,m6_dona_vibrato.wav
3,/Users/jake/code/jchaselubitz/vocal_patterns/v...,Other,straight,m6_caro_straight.wav
4,/Users/jake/code/jchaselubitz/vocal_patterns/v...,Other,straight,m6_row_straight.wav


In [52]:
X = data.drop(columns=['exercise', "technique", "filename"])
y = data[['exercise']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [65]:
y_test[:2677]

,exercise
2677,arpeggios
3282,arpeggios
1555,scales
1414,scales
3419,arpeggios
...,...
2478,arpeggios
1279,scales
281,Other
1387,scales


In [66]:
import sys
sys.path.append('../')

from vocal_patterns.interface.main import predict



In [67]:
X_test["path"].head()

2677    /Users/jake/code/jchaselubitz/vocal_patterns/v...
3282    /Users/jake/code/jchaselubitz/vocal_patterns/v...
1555    /Users/jake/code/jchaselubitz/vocal_patterns/v...
1414    /Users/jake/code/jchaselubitz/vocal_patterns/v...
3419    /Users/jake/code/jchaselubitz/vocal_patterns/v...
Name: path, dtype: object

In [68]:
# X_test["path"][574]
import librosa

waveform, sr = librosa.load(X_test["path"][2555], sr=22050)
X = waveform


In [69]:
import numpy as np
predict(X)
# predicted_indices = np.argmax(predict(X), axis=1)
# predicted_indices


Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk
1/1 [==============================] - 0s 102ms/step


1